In [10]:
import pandas as pd

import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder

In [2]:
# Abrindo o objeto dataframe
with open(r'df-codificado.pickle', 'rb') as pickledfile:
    df = pickle.load(pickledfile)

In [12]:
y = LabelEncoder().fit_transform(X = df['Assunto'].values.reshape(-1,1))

c:\Users\Lucas Alponti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
y

array([3, 3, 3, ..., 0, 0, 0])

In [6]:
# Abrindo o bag-of-words
with open(r'bow.pickle', 'rb') as pickledfile:
    x = pickle.load(pickledfile)

In [15]:
model = RandomForestClassifier()
cv = StratifiedKFold(n_splits = 10)
result = cross_validate(model, x, y, cv = cv, return_estimator=True, return_train_score=True)

In [16]:
result

{'fit_time': array([809.31716967, 755.54001474, 775.33718395, 775.72851467,
        808.54982686, 792.01552534, 787.40804529, 774.25614858,
        744.84567404, 785.46699023]),
 'score_time': array([11.29275179, 11.10722494, 11.09892225, 10.61786509, 10.70986724,
        10.9108572 , 10.29583907, 10.68187094, 10.3858459 , 10.412848  ]),
 'estimator': [RandomForestClassifier(),
  RandomForestClassifier(),
  RandomForestClassifier(),
  RandomForestClassifier(),
  RandomForestClassifier(),
  RandomForestClassifier(),
  RandomForestClassifier(),
  RandomForestClassifier(),
  RandomForestClassifier(),
  RandomForestClassifier()],
 'test_score': array([0.80153378, 0.83908502, 0.84966283, 0.8769007 , 0.94182203,
        0.93203755, 0.93428534, 0.86275288, 0.75247917, 0.83670501]),
 'train_score': array([0.99753184, 0.99665036, 0.99669443, 0.99667974, 0.99615085,
        0.99616554, 0.99609209, 0.99663567, 0.99739962, 0.99651814])}

In [23]:
# Salvando o resultado do random-forest com bow
with open(r'random-forest-bow.pickle', 'wb') as pickledfile:
    pickle.dump(result, pickledfile, protocol=pickle.HIGHEST_PROTOCOL)